# Quantitative Research Task

The purpose of this test is to give you a feel for the analysis of financial time series, and for us assess your ability in programming, machine learning, and statistical modelling.

## 1. Data Processing

The provided data set contains ASX stock prices from Jan 2015 to June 2018. Each csv file contains end of day (EOD) data in the following format: Ticker, Date, Open, High, Low, Close, and Volume.

The first task is converting the EOD data into five seperate time series data frames; one each for Open, High, Low Close and Volume. In each data frame, rows should be indexed by date, and columns by ticker.

In [190]:
import glob
import pandas as pd

# get list of file names from directory
directory = "./ASX-2015-2018/ASX-2015-2018/"
files = glob.glob(directory + '[0-9]*.txt')

# generate list of dataframes from each file (or date) respectively
cl_name = ["ticker", "date", "open", "high", "low", "close", "volume"]
dfs = [pd.read_csv(file, 
                   sep=",", 
                   names = cl_name, 
                   header = None, 
                   parse_dates = ["date"], 
                   index_col = "date") 
       for file in files]

# concatenate the dataframes into one temporary dataframe
df_temp = pd.concat(dfs)

# using pivot function, for each ticker values, individual dataframes for open, high, low, close, and volume can be built
df_open = df_temp[["ticker", "open"]].pivot(columns = "ticker", values = "open")
df_high = df_temp[["ticker", "high"]].pivot(columns = "ticker", values = "high")
df_low = df_temp[["ticker", "low"]].pivot(columns = "ticker", values = "low")
df_close = df_temp[["ticker", "close"]].pivot(columns = "ticker", values = "close")
df_volume = df_temp[["ticker", "volume"]].pivot(columns = "ticker", values = "volume")

Below is the close price dataframe (head) for reference.

In [210]:
df_close.head()

ticker,1AD,1AG,1AL,1PG,1ST,3DM,3DP,3PL,4CE,4DS,...,ZNO,ZNT,ZNZ,ZOZI,ZRL,ZTA,ZUSD,ZYB,ZYL,ZYUS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,1.215,NaN,NaN,NaN,2.26,NaN,NaN,...,NaN,NaN,NaN,NaN,0.067,NaN,NaN,NaN,NaN,NaN
2015-01-05,NaN,NaN,NaN,1.265,NaN,NaN,NaN,2.07,NaN,NaN,...,NaN,NaN,4.40,NaN,0.070,NaN,NaN,NaN,NaN,NaN
2015-01-06,NaN,NaN,NaN,1.245,NaN,NaN,NaN,2.08,NaN,NaN,...,NaN,NaN,4.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,NaN,NaN,NaN,1.250,NaN,NaN,NaN,2.05,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.003,NaN,NaN,NaN,NaN
2015-01-08,NaN,NaN,NaN,1.255,NaN,NaN,NaN,2.08,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next, we create a dataframe containing the future close returns at time $t$, denoted as $r_{t,t+1}$, as defined by:

$$r_{t,t+1}=\frac{P^{c}_{t+1}}{P^{c}_{t}}-1,$$

where $P^c_t$ is the close price at time $t$. This will be the quantity-of-interest that will be predicted.

In [212]:
from pandas.tseries.offsets import BDay

# to find the future close return of t+1, or the close return of next business day, and assign to the index of t
df_ftr_rtn = (df_close.set_index(df_close.index - BDay(1)).iloc[1:] / df_close) - 1
df_ftr_rtn.head()

ticker,1AD,1AG,1AL,1PG,1ST,3DM,3DP,3PL,4CE,4DS,...,ZNO,ZNT,ZNZ,ZOZI,ZRL,ZTA,ZUSD,ZYB,ZYL,ZYUS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,0.041152,NaN,NaN,NaN,-0.084071,NaN,NaN,...,NaN,NaN,NaN,NaN,0.044776,NaN,NaN,NaN,NaN,NaN
2015-01-05,NaN,NaN,NaN,-0.015810,NaN,NaN,NaN,0.004831,NaN,NaN,...,NaN,NaN,-0.002273,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,NaN,NaN,NaN,0.004016,NaN,NaN,NaN,-0.014423,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,NaN,NaN,NaN,0.004000,NaN,NaN,NaN,0.014634,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08,NaN,NaN,NaN,0.015936,NaN,NaN,NaN,0.096154,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Also create a data frame containing close returns for the day, which is calculated with respect to the close price of the previous day.

In [213]:
# to find the future close return of t based on t-1 close values and assign to the index of t
df_rtn = (df_close / df_close.set_index(df_close.index + BDay(1)).iloc[:-1]) - 1
df_rtn.head()

ticker,1AD,1AG,1AL,1PG,1ST,3DM,3DP,3PL,4CE,4DS,...,ZNO,ZNT,ZNZ,ZOZI,ZRL,ZTA,ZUSD,ZYB,ZYL,ZYUS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,NaN,NaN,NaN,0.041152,NaN,NaN,NaN,-0.084071,NaN,NaN,...,NaN,NaN,NaN,NaN,0.044776,NaN,NaN,NaN,NaN,NaN
2015-01-06,NaN,NaN,NaN,-0.015810,NaN,NaN,NaN,0.004831,NaN,NaN,...,NaN,NaN,-0.002273,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,NaN,NaN,NaN,0.004016,NaN,NaN,NaN,-0.014423,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08,NaN,NaN,NaN,0.004000,NaN,NaN,NaN,0.014634,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Create a data frame containing the ratios of $\frac{High}{Low}$ for each ticker each day, in the same format.

In [214]:
df_hl = df_high / df_low
df_hl.head()

ticker,1AD,1AG,1AL,1PG,1ST,3DM,3DP,3PL,4CE,4DS,...,ZNO,ZNT,ZNZ,ZOZI,ZRL,ZTA,ZUSD,ZYB,ZYL,ZYUS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,1.068966,NaN,NaN,NaN,1.055300,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2015-01-05,NaN,NaN,NaN,1.061224,NaN,NaN,NaN,1.111111,NaN,NaN,...,NaN,NaN,1.022727,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2015-01-06,NaN,NaN,NaN,1.041322,NaN,NaN,NaN,1.039604,NaN,NaN,...,NaN,NaN,1.006865,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,NaN,NaN,NaN,1.041667,NaN,NaN,NaN,1.050000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2015-01-08,NaN,NaN,NaN,1.044534,NaN,NaN,NaN,1.014493,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
